## This notebook demonstrates benchmarking the statement extraction pipeline by evaluating the LLM generated statements against the benchmark corpus containing correct INDRA statements. 

Import relevant modules

In [1]:
import sys
from pathlib import Path
# get current path
sys.path.append(str(Path.cwd().parent))

from indra_gpt.benchmarks.benchmark import Benchmark
from indra_gpt.resources.constants import INPUT_DEFAULT


INFO: [2025-02-14 12:16:17] indra.preassembler.grounding_mapper.disambiguate - INDRA DB is not available for text content retrieval for grounding disambiguation.
/Users/thomaslim/miniconda3/envs/indra_gpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's evaluate the performance of OpenAI model 'gpt-4o-mini', with structured_output mode enabled, and for a sample of 10 examples from the benchmark corpus. 

In [2]:
model = 'gpt-4o-mini'
benchmark_file = INPUT_DEFAULT
structured_output = True
n_statements = 10
benchmark = Benchmark(model, benchmark_file, structured_output, n_statements)


In [3]:
benchmark_df = benchmark.get_comparison_df()


Extracting:   0%|          | 0/10 [00:00<?, ?statement/s]INFO: [2025-02-14 12:16:23] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:16:26] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:16:29] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  10%|█         | 1/10 [00:11<01:42, 11.41s/statement]INFO: [2025-02-14 12:16:34] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:16:39] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:16:42] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  20%|██        | 2/10 [00:23<01:36, 12.09s/statement]INFO: [2025-02-14 12:16:46] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I

This generates a dataframe that has input and generated output, along with the comparison results.
Columns:
- <b>'original_statement_json'</b>: The original statement in json format, from the benchmark corpus. 
- <b>'generated_statements_json'</b>: The generated statements in json format, given the input text from the 'evidence' of the original statement.
- <b>'original_statement'</b>: The original statement converted from json format to indra statement. 
- <b>'generated_statements'</b>: The generated statements converted from json format to indra statement.
- <b>'original_statement_grounded'</b>: Grounding process applied to 'original_statement'.
- <b>'generated_statements_grounded'</b>: Grounding process applied to 'generated_statements'.
- <b>'comparison_result'</b>: Result of comparing 'original_statement' with each of the generated statement from 'generated_statements'.
- <b>'comparison_result_grounded'</b>: Same as 'comparison_result' but for 'original_statement_grounded' and 'generated_statements_grounded'.
- <b>'best_match_index'</b>: This is the list index of the 'generated_statements', corresponding to the statement that is most similar with the original statement. 
- <b>'best_match_grounded_index'</b>: Same as 'best_match_index' but for 'generated_statements_grounded'

In [4]:
benchmark_df.head()


,original_statement_json,generated_statements_json,original_statement,generated_statements,original_statement_grounded,generated_statements_grounded,comparison_result,comparison_result_grounded,best_match_index,best_match_grounded_index
0,"{'type': 'Acetylation', 'enz': {'name': 'C5', ...","{""statements"":[{""kind"":""RegulateActivity"",""typ...","[Acetylation(C5(), STAT3())]","[Activation(C5a(), STAT3(mods: (acetylation, K...","[Acetylation(C5(), STAT3())]","[Activation(C5(), STAT3(mods: (acetylation, K,...","[{'equals': False, 'equals_type': False, 'agen...","[{'equals': False, 'equals_type': False, 'agen...",0,0
1,"{'type': 'Acetylation', 'enz': {'name': 'KANSL...","{""statements"":[{""kind"":""Modification"",""type"":""...","[Acetylation(KANSL1(), TP53(), K, 120)]","[Acetylation(MOF(), p53(mods: (acetylation, K,...","[Acetylation(KANSL1(), TP53(), K, 120)]","[Acetylation(MOF(), p53(mods: (acetylation, K,...","[{'equals': False, 'equals_type': True, 'agent...","[{'equals': False, 'equals_type': True, 'agent...",0,0
2,"{'type': 'Acetylation', 'enz': {'name': 'EP300...","{""statements"":[{""kind"":""RegulateActivity"",""typ...","[Acetylation(EP300(), TP73())]",Error: list index out of range,"[Acetylation(EP300(), TP73())]",Error: list index out of range,[{'error': 'One or both lists are empty'}],[{'error': 'One or both lists are empty'}],0,0
3,"{'type': 'Activation', 'subj': {'name': 'BMP',...","{""statements"":[{""kind"":""RegulateAmount"",""type""...","[Activation(BMP(), PTEN())]","[IncreaseAmount(None, PTEN())]","[Activation(BMP(), PTEN())]","[IncreaseAmount(None, PTEN())]","[{'equals': False, 'equals_type': False, 'agen...","[{'equals': False, 'equals_type': False, 'agen...",0,0
4,"{'type': 'Activation', 'subj': {'name': 'Caspa...","{""statements"":[{""kind"":""RegulateActivity"",""typ...","[Activation(Caspase(), PRKCZ())]","[Activation(caspase(), atypical protein kinase...","[Activation(Caspase(), PRKCZ())]","[Activation(Caspase(), atypical protein kinase...","[{'equals': False, 'equals_type': True, 'agent...","[{'equals': False, 'equals_type': True, 'agent...",0,0


## Evaluate the accuracy of the model  

The `benchmark_df` above shows a comprehensive input and output and comparison results for a specified configuration. 
The performance of the model is measured by in terms of three built-in equivalence methods
from `indra` which asks the three questions: 

1) Are the two statements are equal?
2) Do the two statements have the same type?
3) Do the two statements have the same set of agents?

Since the model can extract multiple statements from given input text, we compare the benchmark example with the "best" 
match from the generated set of statements. <br>
E.g. 
- Original statement:  `Activation(BMP(), PTEN())`
- Generated statements: `[Activation(Caspase(), PTEN()), Acetylation(BMP(), PTEN())]` <br><br>
The algorithm prioritizes as follows: statement match > type match > agent set match
<br>So for the above example, `Activation(Caspase(), PTEN())` will be selected over `Acetylation(BMP(), PTEN())` since the types match.

In [5]:
benchmark.compute_comparison_statistics(benchmark_df)


{'config': {'model': 'gpt-4o-mini',
  'benchmark_file': PosixPath('/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/indra_benchmark_corpus_all_correct.json'),
  'structured_output': True,
  'n_statements': 10},
 'equals_accuracy': 0.0,
 'equals_type_accuracy': 0.6,
 'agents_equal_accuracy': 0.1}

* It is very unlikely for 'equals_accuracy' to be a non-zero value, since the two statements are only equal if
types, agent set, and evidence are all equal. 

Benchmark another configuration, this time, 'gpt-4o-mini' but without structured_output mode enabled.

In [15]:
model = 'gpt-4o-mini'
benchmark_file = INPUT_DEFAULT
structured_output = False
n_statements = 10
benchmark = Benchmark(model, benchmark_file, structured_output, n_statements)
benchmark_df = benchmark.get_comparison_df()
benchmark.compute_comparison_statistics(benchmark_df)


Extracting:   0%|          | 0/10 [00:00<?, ?statement/s]INFO: [2025-02-14 12:28:28] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:28:35] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:28:40] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  10%|█         | 1/10 [00:31<04:46, 31.84s/statement]INFO: [2025-02-14 12:28:47] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:28:53] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: [2025-02-14 12:29:00] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Extracting:  20%|██        | 2/10 [00:51<03:19, 24.92s/statement]INFO: [2025-02-14 12:29:07] httpx - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I

{'config': {'model': 'gpt-4o-mini',
  'benchmark_file': PosixPath('/Users/thomaslim/gyorilab/indra_gpt/indra_gpt/resources/indra_benchmark_corpus_all_correct.json'),
  'structured_output': False,
  'n_statements': 10},
 'equals_accuracy': 0.0,
 'equals_type_accuracy': 0.2,
 'agents_equal_accuracy': 0.1}